In [ ]:
# from greem.utility.monitoring import BaseMonitoring, RegularMonitoring, PeriodicMonitoring
from codecarbon import OfflineEmissionsTracker
import os
# import pandas as pd
from time import sleep

In [ ]:
host_name: str = os.uname()[1]
host_name

In [ ]:
mid_res_video: str = 'beauty_mid_res.mp4'
low_res_video: str = 'beauty_low_res.mp4'
high_res_video: str = 'beauty_high_res.mp4'

os.system(
    f'ffmpeg -n -re -i Beauty_s000.265 -c:v:0 libx264 -filter:v:0 "scale=1920:1080" -preset medium {mid_res_video}')
# f'ffmpeg -n -re -i Beauty_s000.265 -c:v:0 libx264 -filter:v:0 "scale=1920:1080" -preset medium {low_res_video}')
os.system(f'ffmpeg -n -re -i Beauty_s000.265 -c:v:0 libx264 -crf 15 -preset medium {high_res_video}')

In [ ]:
# measurement_interval = 0.25
# repetitions = 15
# # idle with just powermeter
# sleep(60)
# 
# # idle with codecarbon
# tracker = OfflineEmissionsTracker(
#     measure_power_secs=measurement_interval,
#     project_name='idle',
#     log_level='error',
#     country_iso_code='AUT')
# 
# tracker.start()
# 
# sleep(60)
# 
# tracker.stop()
# 
# tracker = OfflineEmissionsTracker(
#     measure_power_secs=measurement_interval,
#     project_name=f'high_res_{measurement_interval}s',
#     log_level='error',
#     country_iso_code='AUT')
# 
# tracker.start()
# 
# for i in range(repetitions):
#     os.system(f'ffmpeg -y -re -i {high_res_video} -c:v:0 libx264 -preset medium test.mp4')
# 
# tracker.stop()
# 
# sleep(10)
# 
# tracker = OfflineEmissionsTracker(
#     measure_power_secs=measurement_interval,
#     project_name=f'mid_res_{measurement_interval}s',
#     log_level='error',
#     country_iso_code='AUT')
# 
# tracker.start()
# 
# for i in range(repetitions):
#     os.system(f'ffmpeg -y -re -i {mid_res_video} -c:v:0 libx264 -preset medium test.mp4')
# 
# tracker.stop()